In [1]:
#https://motius.breezy.hr/p/0b622aa1163e01-join-our-tech-community-as-working-student-m-f-d/apply?utm_source=pt-linkedin&utm_medium=job-posting

In [2]:
"""!pip install datasets
!pip install transformers
!pip install sentence_transformers
!pip install peft
!pip install huggingface_hub
!pip install --upgrade accelerate
!pip install --upgrade bitsandbytes
!pip install --upgrade torchmetrics
!pip install --upgrade deepspeed
"""

'!pip install datasets\n!pip install transformers\n!pip install sentence_transformers\n!pip install peft\n!pip install huggingface_hub\n!pip install --upgrade accelerate\n!pip install --upgrade bitsandbytes\n!pip install --upgrade torchmetrics\n!pip install --upgrade deepspeed\n'

In [3]:
from transformers import  AutoModel,AutoTokenizer,AutoModelForMaskedLM , Trainer,TrainingArguments,\
BitsAndBytesConfig,pipeline,default_data_collator,DataCollatorWithPadding,DataCollatorForLanguageModeling
from transformers.utils import move_cache
#from chat_template_utils import get_json_schema
#from llama_cpp import Llama
from utils import *
from peft import *
import datasets
#import torchmetrics
import torch
from torch.utils.data.dataloader import DataLoader,Dataset
#from hqq.engine.hf import HQQModelForCausalLM
#from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
#import deepspeed
import os
import pandas as pd
import json
#from var_dump import var_dump
from bs4 import BeautifulSoup,PageElement,Comment
import matplotlib.pyplot as plt
import seaborn as sns

cache_dir='/var/tmp/.cache/' #'/proj/ciptmp/ix05ogym/.cache/'
output_dir = cache_dir+'outputs/'

move_cache(cache_dir)

def pd_full_screen():
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    
def pd_normal_screen():
    pd.set_option('display.max_rows', 10)
    pd.set_option('display.max_columns', 5)


No ROCm runtime is found, using ROCM_HOME='/usr'
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


0it [00:00, ?it/s]

In [4]:
#torch.cuda.get_device_name()

In [5]:
train_dataset = pd.read_excel('train_dataset.xlsx',index_col=0)
#train_dataset = train_dataset.drop_duplicates()
#train_dataset.to_excel('train_dataset.xlsx')
train_dataset


,query,element,label,action,utterance
0,Email Address,"<input id=""fieldEmail"" name=""cm-ukutgu-ukutgu""...",email,NaN,NaN
1,Accept additional cookies,"<input id=""accept_all_cookies_button"" messaged...",cookie,NaN,NaN
2,Reject additional cookies,"<input id=""accept_essential_cookies_button"" me...",cookie,NaN,NaN
3,Your Name,"<input aria-describedby=""ph-a11y-email-job-nam...",first name,NaN,NaN
4,Recipient’s Email address,"<input aria-describedby="""" aria-label=""Please ...",email,NaN,NaN
...,...,...,...,...,...
2716,og in,"<button class=""careersite-button careersite-bu...",login,1.0,log in
2717,upload resume,"<a href=""javascript: void(0);"" title=""upload r...",cv,1.0,upload resume
2718,upload fileprivacy policy,"<a href=""https://vertriebskarriere.hansemerkur...",privacy,1.0,privacy policy
2719,back,"<a href=""https://bewerber-innendienst.hansemer...",cancel,1.0,back


In [6]:
labeled_forms = pd.read_excel('true_form.xlsx',index_col=0)
labeled_forms

,query,element,label
0,Please enter your personal data below Login \n...,"<form action=""/Vacancies/2224/Application/New/...",application form
1,Werkstudent mwd Forschung und Entwicklung Bew...,"<form action=""/Vacancies/14633/Application/New...",application form
2,Start Application First nameLast nameEmailAppl...,"<form>First name<input label=""First name"" name...",application form
3,Apply 1Personal Details\n \n W...,"<form accept-charset=""UTF-8"" action=""/postings...",application form
4,Career Opportunities Field Supervisor mwd 5296...,"<form accept-charset=""UTF-8"" action=""/career?_...",application form
5,If you have previously started an application ...,"<form action=""https://nottinghamshire.tal.net/...",next
6,Social Sign up\n \n\n\n sign up\n ...,"<form aria-label=""sign up"" novalidate="""" role=...",register form
7,Login Email\n \n\n\...,"<form action=""/cop-data/bd104c9e-a55a-4e72-aa1...",login form
8,Career Opportunities Sign In JavaScript is tur...,"<form accept-charset=""UTF-8"" action=""/career?_...",login form
9,Lets stay connected Register to receive regula...,"<form action=""https://career5.successfactors.e...",newsletter


In [7]:
train_dataset = pd.concat([ train_dataset ,labeled_forms]).reset_index(drop=True)
train_dataset

,query,element,label,action,utterance
0,Email Address,"<input id=""fieldEmail"" name=""cm-ukutgu-ukutgu""...",email,NaN,NaN
1,Accept additional cookies,"<input id=""accept_all_cookies_button"" messaged...",cookie,NaN,NaN
2,Reject additional cookies,"<input id=""accept_essential_cookies_button"" me...",cookie,NaN,NaN
3,Your Name,"<input aria-describedby=""ph-a11y-email-job-nam...",first name,NaN,NaN
4,Recipient’s Email address,"<input aria-describedby="""" aria-label=""Please ...",email,NaN,NaN
...,...,...,...,...,...
2735,Search,"<form name=""searchBox""><input autocomplete=""of...",search form,NaN,NaN
2736,Search by Keyword\n\n\n\nShow More Options\n\n...,"<form action=""/search/"" lang=""en-US"" method=""g...",search form,NaN,NaN
2737,Keywords\n\nLocation\n\n\nSearch Jobs,"<form action=""/en/search-jobs"" data-ambient-se...",search form,NaN,NaN
2738,Search by Keyword\n\n\n\n\n\nLoading\n\n\n\n ...,"<form action=""/search/"" lang=""en-US"" method=""g...",search form,NaN,NaN


In [8]:
train_dataset = train_dataset.fillna('')
train_dataset

,query,element,label,action,utterance
0,Email Address,"<input id=""fieldEmail"" name=""cm-ukutgu-ukutgu""...",email,,
1,Accept additional cookies,"<input id=""accept_all_cookies_button"" messaged...",cookie,,
2,Reject additional cookies,"<input id=""accept_essential_cookies_button"" me...",cookie,,
3,Your Name,"<input aria-describedby=""ph-a11y-email-job-nam...",first name,,
4,Recipient’s Email address,"<input aria-describedby="""" aria-label=""Please ...",email,,
...,...,...,...,...,...
2735,Search,"<form name=""searchBox""><input autocomplete=""of...",search form,,
2736,Search by Keyword\n\n\n\nShow More Options\n\n...,"<form action=""/search/"" lang=""en-US"" method=""g...",search form,,
2737,Keywords\n\nLocation\n\n\nSearch Jobs,"<form action=""/en/search-jobs"" data-ambient-se...",search form,,
2738,Search by Keyword\n\n\n\n\n\nLoading\n\n\n\n ...,"<form action=""/search/"" lang=""en-US"" method=""g...",search form,,


In [28]:
c = train_dataset['element'].str.contains('<select')
train_dataset[c]

,query,element,label,action,utterance
909,,"<select name=""gender"">\n<option value="""">selec...",1,,male
910,,"<select name=""gender"">\n<option value="""">selec...",2,,female
911,,"<select name=""gender"">\n<option value="""">selec...",3,,non-binary
912,,"<select name=""gender"">\n<option value="""">selec...",4,,prefer not to say
913,,"<select name=""language"">\n<option value="""">sel...",1,,english
...,...,...,...,...,...
2724,Apply 1Personal Details\n \n W...,"<form accept-charset=""UTF-8"" action=""/postings...",application form,,
2725,Career Opportunities Field Supervisor mwd 5296...,"<form accept-charset=""UTF-8"" action=""/career?_...",application form,,
2733,AKKA job opening search engine\n \n...,"<form action=""./stelle-softwareentwickler-c-c-...",search form,,
2736,Search by Keyword\n\n\n\nShow More Options\n\n...,"<form action=""/search/"" lang=""en-US"" method=""g...",search form,,


In [9]:
test_dataset = pd.read_excel('./data/forms.xlsx')
c = test_dataset['label'].notna()
test_dataset = test_dataset.loc[c,:].reset_index(drop=True)
test_dataset

,query,element,label,sudo_label
0,Personal information\n CV or resume\n Browse C...,"<input accept=""application/pdf,.pdf"" name=""cv""...",cv,login
1,Name *\n,"<input name=""first_name"" placeholder=""First"" r...",first name,first name
2,This field contains invalid characters\n,"<input name=""last_name"" placeholder=""Last"" req...",last name,transcript
3,This field contains invalid characters\n Email...,"<input name=""email"" placeholder=""yourmail@doma...",email,email
4,Phone\n,"<input name=""phone"" placeholder=""+49 176 123 4...",phone number,phone number
...,...,...,...,...
95,cover letter\n upload your cover letter\n,"<input accept=""application/msword, application...",cover letter,NaN
96,github\n,"<input name=""custom_attribute_356304"" placehol...",github,NaN
97,ihre e-mail-adresse\n,"<button aria-label=""subscribe for newsletter"" ...",newsletter,NaN
98,(!)\n this field is required\n,"<button id=""uploaddropboxresume"" type=""submit""...",submit,NaN


In [10]:

person = {
    "first name": "majid",
    "last name": "sharghi foroushani",
    "pronoun": "he",
    "birth": "10/08/1997",
    "age": "27",
    "sex": "male",
    "skill": "NLP,LLM",
    "country": "Germany",
    "address": "Philip-9",
    "city": "Erlangen",
    "phone number": "+497773154",
    "available from": "6/24/2024",
    "salary": "10000",
    "German language level": "A1",
    "English language level": "C1",
    "visa": "I do not need visa.",
    "email": "sharghi.majid@gmail.com",
    "password": "123456",
    "find us": "Google",
    "xing": "www.xing.com",
    "job title": "I am a stduent.",
    "university": "FAU",
    "major": "AI",
    "privacy": "privacy",
    "search": "search",
    "submit": "submit",
    "cookie": "cookie",
    "cover letter": "cover letter",
    "linkedin": "www.linkedin.com",
    "github": "www.github.com",
    "login": "login",
    "register": "register",
    "cv": "/pah/to/cv.pdf",
    "twitter": "www.twitter.com",
    "years of experience": "4 years in deep learning",
    "username":"Majidsh97",
    "website language":"english",
    "transcript":"path/to/ts.pdf",
    "company":"company",
    "house number":"9",
    "italian language level":"A2",
    "ethnicity":"white",
    "picture":"path/to/p.jpg",
    "career level":"entry",
    "zip":"91052",
    "newsletter":"newsletter",
    "fake":"fake",
    "question":"question",
    "cancel":"cancel",
    "recommender":"my recommender",
    "website":"my website",
    'workherebefore':"No",
    "next":"next",
    "dropbox":"www.dropbox.com",
    "googledrive":"googledrive",
    "work hours":"8",
    "workduration":"6 months",
    "willing to relocate":"yes",
    "universityenrollment":"path to enrollment letter",
    "subscribe":"subscribe",
    "distance":"50 km",
    "captcha":"captcha",
    "forgot password":"no",
    "graduation year":"2024",
    "civic number":"54864",
    "person title":"Mr",
    "show password":"show password",
    
    
}
# {'years of experience', 'forgot password', 'next', 'username', 'website language', 'latitude', 'distance', 'transcript', 'english language level', 'cancel', 'company', 'longitude', 'house number', 'italian language level', 'title', 'captcha', 'civic number', 'zip', 'recommender', 'newsletter', 'show password', 'ethnicity', 'graduation year', 'job title', 'subscribe', 'picture', 'career level'}




In [11]:
from transformers import Qwen2ForCausalLM,T5ForConditionalGeneration
device = 'cuda'
model_name = 'Qwen/Qwen2-0.5B-Instruct'
#model_name = "google/flan-t5-small"
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=cache_dir)
model = Qwen2ForCausalLM.from_pretrained(model_name #Qwen/Qwen2-0.5B-Instruct
                                         #,attn_implementation="flash_attention_2"
                                         #,torch_dtype=torch.float16
                                         ,quantization_config=nf4_config
                                         , device_map='auto',cache_dir=cache_dir)#.to('cuda')

model.eval()
#model.config['use_sliding_window']=True
print(model.config)

model

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Qwen2Config {
  "_name_or_path": "Qwen/Qwen2-0.5B-Instruct",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "max_position_embeddings": 32768,
  "max_window_layers": 24,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 1e-06,
  "rope_theta": 1000000.0,
 

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear4bit(in_features=896, out_features=896, bias=True)
          (k_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (v_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (o_proj): Linear4bit(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear4bit(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
 

In [12]:
model.generation_config

GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "repetition_penalty": 1.1,
  "temperature": 0.7,
  "top_k": 20,
  "top_p": 0.8
}

In [13]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model,LoraConfig(task_type=TaskType.CAUSAL_LM,target_modules=['q_proj','v_proj','k_proj']))
#model = get_peft_model(model,LoraConfig(task_type=TaskType.CAUSAL_LM,target_modules=['q','v','k']))

model.print_trainable_parameters()
model
for n,p in model.named_parameters():
    if p.requires_grad==False:
        print(n)

trainable params: 737,280 || all params: 494,770,048 || trainable%: 0.1490
base_model.model.model.embed_tokens.weight
base_model.model.model.layers.0.self_attn.q_proj.base_layer.weight
base_model.model.model.layers.0.self_attn.q_proj.base_layer.bias
base_model.model.model.layers.0.self_attn.k_proj.base_layer.weight
base_model.model.model.layers.0.self_attn.k_proj.base_layer.bias
base_model.model.model.layers.0.self_attn.v_proj.base_layer.weight
base_model.model.model.layers.0.self_attn.v_proj.base_layer.bias
base_model.model.model.layers.0.self_attn.o_proj.weight
base_model.model.model.layers.0.mlp.gate_proj.weight
base_model.model.model.layers.0.mlp.up_proj.weight
base_model.model.model.layers.0.mlp.down_proj.weight
base_model.model.model.layers.0.input_layernorm.weight
base_model.model.model.layers.0.post_attention_layernorm.weight
base_model.model.model.layers.1.self_attn.q_proj.base_layer.weight
base_model.model.model.layers.1.self_attn.q_proj.base_layer.bias
base_model.model.model

In [14]:
import random


def get_random():
    d = {}
    for k in person.keys():
        d[k] = str(random.randint(1000, 10000))

    return d


# get_random()
def dict_to_str(p):
    s = "{"
    for k, v in p.items():
        s += f'"{k}":"{v}",'
        pass

    s += "}"
    return s


def get_one_chat_template(query,element,utterance="",person_str="", label_str=None,label_action=None,action_history=""):
    d = [
        {   #{"role":"system","content":""}
            "role": "user",
            #"content": """Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""
            "content": """Your answer has to be in short JSON format. Use this CONTENT in your answer:\n"""+
            f"""CONTENT: {{"{utterance}",{person_str}}}."""

            #f"\ncontent: {person_str}"
            
        },
        {
            "role": "user",
            #"content": "What is this HTML element about?\n" + query_element_str,
            #"content": "Question:\n" + query_element_str,
            
            "content": f"{query}"+
            f"\n{element}"
            #f"\nutterance: {utterance}"  
        },
    ]
    if label_str:
        d.append(
            {"role": "assistant", "content": '{"answer":"' +str( label_str )+ '","click":"'+str(label_action)+'"}'},
        )
        
    return d


def get_one_msg(text, label):
    p = person #get_random()
    strp = ""#dict_to_str(p)
    rc = ""  # random.choice(list(p.keys()))+' '
    d = get_one_chat_template(strp, rc + text,label)#p[label])
    return d


def get_chat():

    d=train_dataset.apply( lambda x :get_one_chat_template(x['query'],x['element'],x['utterance'],"",x['label'],x['action']),axis=1).tolist()

    return d  # ,labels


get_chat()

[[{'role': 'user',
   'content': 'Your answer has to be in short JSON format. Use this CONTENT in your answer:\nCONTENT: {"",}.'},
  {'role': 'user',
   'content': 'Email Address\n<input id="fieldEmail" name="cm-ukutgu-ukutgu" required="" type="email"/>'},
  {'role': 'assistant', 'content': '{"answer":"email","click":""}'}],
 [{'role': 'user',
   'content': 'Your answer has to be in short JSON format. Use this CONTENT in your answer:\nCONTENT: {"",}.'},
  {'role': 'user',
   'content': 'Accept additional cookies\n<input id="accept_all_cookies_button" messagedays="90" name="accept_all_cookies_button" type="submit" value="Accept additional cookies"/>'},
  {'role': 'assistant', 'content': '{"answer":"cookie","click":""}'}],
 [{'role': 'user',
   'content': 'Your answer has to be in short JSON format. Use this CONTENT in your answer:\nCONTENT: {"",}.'},
  {'role': 'user',
   'content': 'Reject additional cookies\n<input id="accept_essential_cookies_button" messagedays="90" name="accept_ess

In [15]:
def get_train_data():
    return tokenizer.apply_chat_template(get_chat(),add_generation_prompt=False ,truncation=True,max_length=512)  # ,return_tensors='pt')
    pass

tokenized_data = get_train_data()


no_label = tokenizer.apply_chat_template([  get_one_chat_template( 
                                                                   str(x['query']),
                                                                   x['element']
                                                                   ) 
                                for i,x in test_dataset.iterrows()] ,truncation=True,max_length=512)  # ,return_tensors='pt')

print(len(tokenized_data))
print(tokenizer.decode(token_ids=tokenized_data[0]))
print(len(tokenized_data[0]))
print(tokenizer.decode(token_ids=no_label[0]))


2740
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Your answer has to be in short JSON format. Use this CONTENT in your answer:
CONTENT: {"",}.<|im_end|>
<|im_start|>user
Email Address
<input id="fieldEmail" name="cm-ukutgu-ukutgu" required="" type="email"/><|im_end|>
<|im_start|>assistant
{"answer":"email","click":""}<|im_end|>

83
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Your answer has to be in short JSON format. Use this CONTENT in your answer:
CONTENT: {"",}.<|im_end|>
<|im_start|>user
Personal information
 CV or resume
 Browse CV to autocomplete your application. You can still fill your profile manually.
 
<input accept="application/pdf,.pdf" name="cv" tabindex="-1" type="file"/><|im_end|>



In [16]:
len(tokenized_data[0])

83

In [17]:
#tokenizer(['<|im_start|>assistant'])
tokenizer(['assistant'])


{'input_ids': [[77091]], 'attention_mask': [[1]]}

In [18]:
mydc = DataCollatorForLanguageModeling(tokenizer,False)
def mmf(x):
    y = mydc(x)
    b = (y['labels']==77091).int().argmax()-1
    for i in range(0,b):
        y['labels'][0][i]=-100
        pass
    #print( y['labels'] )
    return y



In [19]:
from transformers import TrainerCallback
"""
class ChangeDatasetCallback(TrainerCallback):
    def __init__(self):
        #super.__init__()
        pass

    def on_epoch_end(self, args, state, control, **kwargs):
        # Move to the next dataset at the end of each epoch
        #self.current_epoch += 1
        #new_dataset = self.datasets[self.current_epoch % len(self.datasets)]
        kwargs['train_dataloader'].dataset = get_train_data()
        #kwargs['eval_dataloader'].dataset = new_dataset['validation']
"""

class custom_dataset(Dataset):
    def __init__(self) -> None:
        super().__init__()
        self.dataset = get_train_data()
        self.len = len(self.dataset)
        pass

    def __getitem__(self, i) :
        if i==self.len-1:
            self.dataset = get_train_data()

        return self.dataset[i]
        pass

    def __len__(self):
        return self.len
        pass

trainer =  Trainer(model,
        train_dataset=custom_dataset(),
        data_collator=mmf,
        #compute_metrics=compute_metric,
        #callbacks=[ChangeDatasetCallback()],
        args=TrainingArguments(
          output_dir=output_dir,
          num_train_epochs=2,
          per_device_train_batch_size=1,
          per_device_eval_batch_size=1,
          logging_steps=300,

          #eval_accumulation_steps =2,
            #optim="paged_adamw_8bit"
          


        ))

trainer.train()
model = model.eval()

[2024-07-12 08:51:53,372] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


Step,Training Loss
300,1.035700
600,0.351500
900,0.308300
1200,0.232700
1500,0.242100
1800,0.223600
2100,0.212500
2400,0.169700
2700,0.202100
3000,0.142300


/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: F

In [26]:
trainer.model.save_pretrained('./saved/q79')#output_dir+'saved_q4/')


/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [21]:
questions = [
"use html to create email"
,"""what is your gender?"""
,"""where are you living?(optional)"""
,"what is your salary expectation?*"
,"Which pronouns should we use to address you? (optional)"
,"how is your German?"
,"do you need visa spunsership?"
,"where are you working right now?"
,"what is your birth date?"
,"what are your hobbies?"
,"password"
,"what projects have you done?"
,"are you a text ai?"
,"what experice do you have in network engineering?"
,"what projects have you done in deep learning?"
,"how did you find us?"]

#model.disable_adapter()

x= [get_one_chat_template(#person_to_str(person)
                          q,"""<input type="text" style="" id="custom">""") for q in questions]
tokenized_questions = tokenizer.apply_chat_template(x)

with torch.no_grad():
    
    for i,x in enumerate(tokenized_questions):
            x = torch.tensor([x]).to('cuda')
            o = model.generate(input_ids=x,max_new_tokens=500,do_sample=False)
            
            print(questions[i])
            #decoded = tokenizer.decode(o[0,x.shape[1]+3:])
            decoded = tokenizer.decode(o[0])
            
            print(decoded)
            print('--------------------------------------------------------')
        
    

/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:537: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


use html to create email
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Your answer has to be in short JSON format. Use this CONTENT in your answer:
CONTENT: {"",}.<|im_end|>
<|im_start|>user
use html to create email
<input type="text" style="" id="custom"><|im_end|>
<|im_start|>assistant
{"answer":"email","click":""}<|im_end|>
--------------------------------------------------------
what is your gender?
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Your answer has to be in short JSON format. Use this CONTENT in your answer:
CONTENT: {"",}.<|im_end|>
<|im_start|>user
what is your gender?
<input type="text" style="" id="custom"><|im_end|>
<|im_start|>assistant
{"answer":"gender","click":""}<|im_end|>
--------------------------------------------------------
where are you living?(optional)
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Your answer has to be in short JSON format. Use this CONTENT in your a

In [24]:
model = model.to('cuda')
def get_answer(decoded):
    try:
                #print("ERROR: ",decoded)

                _s = decoded.find('{')
                _e = decoded.find('}',_s)+1
                #print(decoded[_s:_e])
                answer = json.loads(decoded[_s:_e])['answer']
                return answer
                
    except Exception as e:
                print("ERROR: ",e)
                print(decoded)
                print("---------------------------------")

                
def compute_metric():
    model.eval()
    acc=0
    with torch.no_grad():
        for i in range(len(no_label)):
            x = mydc([no_label[i]])['input_ids'].to('cuda')
            #print(x)
            o = model.generate(input_ids=x,max_new_tokens=20,do_sample=False)
            decoded = tokenizer.decode(o[0,x.shape[1]:])
            #print(decoded)
            #break

            #print(x[0],x[1])
            answer = get_answer(decoded)
            _label = test_dataset.loc[i,'label']
            #_label = person[test_dataset.loc[i,'label']]
            c = answer==_label

            if c:
                acc +=1
            else:
                print("NO_CORRECT:",answer,_label)
                print("---------------------------------")

                pass

    acc/=len(no_label)
    print("acc:",acc)



compute_metric()

NO_CORRECT: cv privacy
---------------------------------
NO_CORRECT: 1 sex
---------------------------------
ERROR:  Expecting value: line 1 column 1 (char 0)
 value="country_chad"> Chad</option><option value="country_chile"> Chile</option
---------------------------------
NO_CORRECT: None country
---------------------------------
NO_CORRECT: cv cover letter
---------------------------------
NO_CORRECT: 1 sex
---------------------------------
NO_CORRECT: 1 visa
---------------------------------
NO_CORRECT: gender sex
---------------------------------
NO_CORRECT: search submit
---------------------------------
NO_CORRECT: cover image cover letter
---------------------------------
NO_CORRECT: linkedin xing
---------------------------------
NO_CORRECT: cv submit
---------------------------------
NO_CORRECT: 2 submit
---------------------------------
NO_CORRECT: 5 country
---------------------------------
NO_CORRECT: 2 country
---------------------------------
ERROR:  Expecting value: line

In [23]:
model.eval()
i =11
d = mydc(no_label[i:i+1])['input_ids'].to(device)
d = tokenizer.apply_chat_template( [#{"role":"system","content":""}
                             {"role":"user", "content":"""Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""+s}
                            ,{"role":"user", "content":'email'}] , return_tensors='pt').to(device)
#d = tokenizer(['json {"answer":""} what is this html element about? <input type="email">'],return_tensors='pt')['input_ids'].to(device)
#print(d)
with torch.no_grad():

    o = model.generate(input_ids = d,max_length = 512,do_sample=True)
    print(tokenizer.decode(o[0]))



NameError: name 's' is not defined

In [ ]:
"""x =Qwen2ForCausalLM.from_pretrained('Qwen/Qwen2-0.5B-Instruct',cache_dir=cache_dir)
model = PeftModelForCausalLM.from_pretrained(x,output_dir+'saved/')"""